# Positive Predictive Value Analysis for Imaging Modalities

## Introduction to PPV in Medical Imaging

Positive Predictive Value (PPV) is a key metric in medical imaging that tells us how reliable a positive test result is. It answers the question: "When the test says there's something there, how often is it actually there?"

PPV is calculated as:
```
PPV = (True Positives) / (True Positives + False Positives)
```

- True Positives: Exact, Side, or Partial matches
- False Positives: Incorrect Localization
- Outputs total localized scans, match counts, and PPV %.

PPV works together with the other metrics we have:
- Sensitivity tells us how good a test is at finding actual problems
- Modality concordance looks at when different imaging methods agree with each other

For parathyroid imaging, high PPV means we can trust positive findings to guide surgery. When multiple imaging methods agree (concordance), it often increases our confidence in the results.


Import

In [1]:
# Import needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Reading Data

In [2]:
# Read the data
data = pd.read_excel("Data/Clean_imaging_data.xlsx")

In [4]:
# Print column names and first few rows
print("Column names:")
print(data.columns.tolist())
print("\nFirst few rows:")
print(data.head())

Column names:
['Date of Procedure', 'Patient ID', 'Age', 'Gender', 'Surgical Findings', 'Surgical Cure', 'SPECT/CT', 'Ultrasound', '4D CT Scan', 'Sestamibi', 'MRI', 'SPECT/CT Calculated Concordance', '4D CT Scan Calculated Concordance', 'Ultrasound Calculated Concordance', 'Sestamibi Calculated Concordance', 'MRI Calculated Concordance', 'Surgical Side']

First few rows:
  Date of Procedure  Patient ID  Age Gender              Surgical Findings  \
0        2012-04-24      156895   77      F                 Right Inferior   
1        2012-05-08      182059   70      F                  Left Inferior   
2        2012-05-22      150593   65      F                  Left Inferior   
3        2012-07-03      183346   69      F  Left Superior, Right Superior   
4        2012-07-18      183255   74      F                  Left Superior   

  Surgical Cure SPECT/CT      Ultrasound 4D CT Scan       Sestamibi  \
0           Yes      NaN             NaN        NaN  Right Inferior   
1           Yes

## 1. SPECT/CT Analysis

Analysis Steps:
1. Filter data for SPECT/CT results
2. Count true positives (correctly identified glands) and false positives (incorrect identifications)
3. Calculate PPV to see how reliable positive SPECT/CT findings are
4. Create visualization of results


In [3]:
# SPECT/CT PPV Analysis

# Filter for SPECT/CT scans
spect_ct_data = data[data['imaging_type'] == 'SPECT/CT']  # adjust column name as per your data

# Count matches
true_positives = len(spect_ct_data[spect_ct_data['match_type'].isin(['Exact', 'Side', 'Partial'])])
total_localized = len(spect_ct_data[spect_ct_data['localization_status'] == 'Localized'])  # adjust as needed

# Calculate PPV
spect_ct_ppv = (true_positives / total_localized) * 100 if total_localized > 0 else 0

# Create summary
print(f"SPECT/CT Analysis:")
print(f"Total Localized Scans: {total_localized}")
print(f"True Positives: {true_positives}")
print(f"PPV: {spect_ct_ppv:.2f}%")

# Visualization
plt.figure(figsize=(8, 6))
plt.bar(['True Positives', 'False Positives'], 
        [true_positives, total_localized - true_positives])
plt.title('SPECT/CT PPV Breakdown')
plt.ylabel('Number of Cases')
plt.show()


KeyError: 'imaging_type'

## 2. SPECT/CT + Sestamibi Concordance Analysis

Analysis Steps:
1. Find cases where both SPECT/CT and Sestamibi tests were performed
2. Identify concordant results (when both tests point to the same location)
3. Calculate PPV for these concordant cases to see if agreement improves accuracy
4. Create visualization comparing concordant vs. single-modality results


In [ ]:
# SPECT/CT + Sestamibi Concordance PPV Analysis

# Find cases where both tests were done
concordant_cases = data[
    (data['has_spect_ct'] == True) & 
    (data['has_sestamibi'] == True)  # adjust column names
]

# Find cases where they agree
concordant_positive = concordant_cases[
    (concordant_cases['spect_ct_location'] == concordant_cases['sestamibi_location']) &
    (concordant_cases['localization_status'] == 'Localized')
]

# Calculate PPV for concordant cases
true_positives = len(concordant_positive[concordant_positive['match_type'].isin(['Exact', 'Side', 'Partial'])])
total_concordant = len(concordant_positive)

concordant_ppv = (true_positives / total_concordant) * 100 if total_concordant > 0 else 0


## 3. Sestamibi Analysis

Analysis Steps:
1. Isolate Sestamibi scan results from our dataset
2. Count true positives (confirmed parathyroid tissue) and false positives (other tissue types)
3. Calculate PPV to determine how often positive Sestamibi findings are correct
4. Create visualization of Sestamibi performance


In [ ]:
# Sestamibi PPV Analysis

# Same structure but filter for Sestamibi
sestamibi_data = data[data['imaging_type'] == 'Sestamibi']  # adjust column name

# Count matches
true_positives = len(sestamibi_data[sestamibi_data['match_type'].isin(['Exact', 'Side', 'Partial'])])
total_localized = len(sestamibi_data[sestamibi_data['localization_status'] == 'Localized'])

# Calculate PPV
sestamibi_ppv = (true_positives / total_localized) * 100 if total_localized > 0 else 0

# Print results and create similar visualization


## 4. 4D CT Analysis

Analysis Steps:
1. Extract all 4D CT results from our dataset
2. Identify true positives (actual parathyroid glands) and false positives (other tissues)
3. Calculate PPV to assess the reliability of positive 4D CT findings
4. Create visualization showing 4D CT performance metrics


In [ ]:
# 4D CT PPV Analysis

# Filter for 4D CT
ct_data = data[data['imaging_type'] == '4D CT']  # adjust column name

# Same counting and calculation as above


## 5. Combined Analysis of All Modalities

Analysis Steps:
1. Collect PPV results from all imaging methods (SPECT/CT, Sestamibi, 4D CT, and concordant findings)
2. Create a comparison table showing PPV, sample sizes, and confidence intervals
3. Make a bar chart to visually compare PPVs across methods
4. Perform statistical tests to see if some methods are significantly better than others
5. Write a summary of key findings and potential clinical implications


In [ ]:
# Create comparison dataframe
ppv_comparison = pd.DataFrame({
    'Imaging Method': ['SPECT/CT', 'Sestamibi', '4D CT', 'Concordant'],
    'PPV': [spect_ct_ppv, sestamibi_ppv, ct_ppv, concordant_ppv],
    'Total Cases': [spect_ct_total, sestamibi_total, ct_total, concordant_total],
    'True Positives': [spect_ct_true_pos, sestamibi_true_pos, ct_true_pos, concordant_true_pos]
})

# Create comparison visualization
plt.figure(figsize=(10, 6))
plt.bar(ppv_comparison['Imaging Method'], ppv_comparison['PPV'])
plt.title('PPV Comparison Across Imaging Methods')
plt.ylabel('PPV (%)')
plt.xticks(rotation=45)
plt.show()

# Statistical testing
# You might want to use chi-square test to compare proportions
from scipy.stats import chi2_contingency
# Add statistical testing code here